# 冒烟测试

快速验证 holdem-lab 已正确安装并能正常工作。

In [1]:
# 导入所有模块
from holdem_lab import (
    Card, Deck, Rank, Suit,
    parse_card, parse_cards, format_cards,
    HandType, HandRank, evaluate_hand, find_winners,
    EquityRequest, PlayerHand, calculate_equity,
    EventLog, EventType, HandReplayer,
    GameState, Street,
)
print("所有模块导入成功！")

所有模块导入成功！


## 1. 卡牌解析

In [2]:
# 解析卡牌
cards = parse_cards("Ah Kh Qh Jh Th")
print(f"解析结果: {format_cards(cards)}")
print(f"花色显示: {' '.join(c.pretty() for c in cards)}")

解析结果: Ah Kh Qh Jh Th
花色显示: A♥ K♥ Q♥ J♥ T♥


## 2. 手牌评估

In [3]:
# 评估不同手牌
test_hands = [
    ("Ah Kh Qh Jh Th", "Royal Flush"),
    ("9h 8h 7h 6h 5h", "Straight Flush"),
    ("Ah Ad Ac As Kh", "Four of a Kind"),
    ("Ah Ad Ac Kh Kd", "Full House"),
    ("Ah Kh Qh Jh 9h", "Flush"),
    ("Ah Kd Qc Js Th", "Straight"),
    ("Ah Ad Ac Kh Qd", "Three of a Kind"),
    ("Ah Ad Kh Kd Qc", "Two Pair"),
    ("Ah Ad Kh Qd Jc", "One Pair"),
    ("Ah Kd Qc Js 9h", "High Card"),
]

print("手牌评估测试:")
print("-" * 50)
for cards_str, expected in test_hands:
    cards = parse_cards(cards_str)
    rank = evaluate_hand(cards)
    status = "✓" if expected in str(rank.hand_type) else "✗"
    print(f"{status} {cards_str:25} -> {rank.describe()}")

手牌评估测试:
--------------------------------------------------
✓ Ah Kh Qh Jh Th            -> Royal Flush
✓ 9h 8h 7h 6h 5h            -> Straight Flush, Nine high
✗ Ah Ad Ac As Kh            -> Four of a Kind, Aces
✓ Ah Ad Ac Kh Kd            -> Full House, Aces full of Kings
✓ Ah Kh Qh Jh 9h            -> Flush, Ace high
✓ Ah Kd Qc Js Th            -> Straight, Ace high
✗ Ah Ad Ac Kh Qd            -> Three of a Kind, Aces
✓ Ah Ad Kh Kd Qc            -> Two Pair, Aces and Kings
✓ Ah Ad Kh Qd Jc            -> Pair of Aces
✓ Ah Kd Qc Js 9h            -> High Card, Ace


## 3. 权益计算

In [4]:
import time

# 经典对局：AA vs KK
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(parse_cards("Ah Ad"))),
        PlayerHand(hole_cards=tuple(parse_cards("Kh Kd"))),
    ],
    num_simulations=10000,
    seed=42,
)

start = time.perf_counter()
result = calculate_equity(request)
elapsed = time.perf_counter() - start

print("AA vs KK:")
print(f"  AA 权益: {result.players[0].equity:.1%}")
print(f"  KK 权益: {result.players[1].equity:.1%}")
print(f"  (预期: ~82% vs ~18%)")
print(f"  执行时间: {elapsed:.3f}s ({request.num_simulations} 次模拟)")

AA vs KK:
  AA 权益: 82.1%
  KK 权益: 17.9%
  (预期: ~82% vs ~18%)
  执行时间: 1.369s (10000 次模拟)


## 4. 游戏状态

In [5]:
# 运行完整牌局
game = GameState(num_players=3, seed=42)
result = game.run_to_showdown()

print("牌局结果:")
print(f"  公共牌: {format_cards(game.board)}")
for i, player in enumerate(game.players):
    print(f"  玩家 {i}: {format_cards(player.hole_cards)}")
print(f"  获胜者: 玩家 {result.winners[0]}")
print(f"  获胜牌型: {result.winning_hand.describe()}")

牌局结果:
  公共牌: 6c Js 6s 4s Kh
  玩家 0: 4d 7s
  玩家 1: 8d 2s
  玩家 2: 7d Jh
  获胜者: 玩家 2
  获胜牌型: Two Pair, Jacks and Sixs


## 5. 事件日志

In [6]:
# 查看事件日志
log = game.event_log
print(f"事件总数: {len(log)}")
print("\n事件类型:")
for event in log:
    print(f"  {event.event_type.value}")

事件总数: 9

事件类型:
  init_hand
  deal_hole
  deal_hole
  deal_hole
  deal_flop
  deal_turn
  deal_river
  showdown
  payout


## 所有测试通过！

如果你在上方看到了对勾 (✓) 和合理的输出，说明 holdem-lab 工作正常。